In [1]:
import sys

In [2]:
sys.path.append('C:\\Users\\gertj\\OneDrive\\Bureaublad\\School\\Master\\Thesis\\RobustConAcq\\RobustConAcq')

In [3]:
from pycona import *
import cpmpy as cp

In [4]:
from pycona.active_algorithms import RobustAcq
from pycona.answering_queries import MisclassifyingOracle

In [5]:
def construct_nurse_rostering(shifts_per_day, num_days, num_nurses, nurses_per_shift):

    # Define the variables
    roster_matrix = cp.intvar(1, num_nurses, shape=(num_days, shifts_per_day, nurses_per_shift), 
                              name="shifts")


    # Create the language:
    AV = absvar(2)  # create abstract vars - as many as maximum arity
    # create abstract relations using the abstract vars
    lang = [AV[0] == AV[1], AV[0] != AV[1], AV[0] < AV[1], AV[0] > AV[1], AV[0] >= AV[1], AV[0] <= AV[1]]

    instance = ProblemInstance(variables=roster_matrix, language=lang)

    return instance

In [6]:
def construct_nurse_rostering_with_oracle(shifts_per_day, num_days, num_nurses, nurses_per_shift):
    
    inst = construct_nurse_rostering(shifts_per_day, num_days, num_nurses, nurses_per_shift)
    
    # Create an oracle for simulation, by defining the ground truth problem
    ## Define the variables
    roster_matrix = inst.variables
    
    ## Define the list of (fixed-arity/decomposed) constraints
    C_T = []

    ## Constraint: Each shift in a day must be assigned to a different nurse
    for day in range(num_days):
        C_T += cp.AllDifferent(roster_matrix[day, ...]).decompose()   # ... means all remaining dimensions

    ## Constraint: The last shift of a day cannot have the same nurse as the first shift of the next day
    for day in range(num_days - 1):
        C_T += cp.AllDifferent(roster_matrix[day, shifts_per_day - 1], roster_matrix[day + 1, 0]).decompose()

    oracle = MisclassifyingOracle(C_T, misclassification_rate=0)

    return inst, oracle

In [7]:
instance, oracle = construct_nurse_rostering_with_oracle(3, 2, 8, 2)

In [8]:
ca = RobustAcq()
learned_instance = ca.learn(instance, oracle=oracle, verbose=1)

396
0
396
q1
random: 0.30375651534656756
.q1 scope
random: 0.4843105151131195
.random: 0.8536474255381581
.random: 0.7948820632143154
.random: 0.2199365137358772
.[(shifts[0,2,1]) > (shifts[1,0,1]), (shifts[0,2,1]) < (shifts[1,0,1]), (shifts[0,2,1]) != (shifts[1,0,1])]
random: 0.4489776705749208
.random: 0.27998602237711634
.q1 cl
L390
6
396
q1
random: 0.06317122814317067
.q1 scope
random: 0.5193897514580984
.random: 0.8087556047326848
.random: 0.8224555253168508
.random: 0.01919977468212486
.random: 0.8851691982572087
.random: 0.9252703565515094
.[(shifts[0,0,0]) != (shifts[0,2,1]), (shifts[0,0,0]) < (shifts[0,2,1]), (shifts[0,0,0]) > (shifts[0,2,1])]
random: 0.08660714974804884
.random: 0.8937387025637424
.q1 cl
L377
19
396
q1
random: 0.14318079410069495
.q1 scope
random: 0.256471839066708
.random: 0.9176258887396966
.random: 0.5518761736418272
.random: 0.9320870414509228
.random: 0.4798436069923777
.random: 0.15400654093841415
.random: 0.6570298601962865
.[(shifts[1,1,0]) < (shifts[

In [9]:
learned_instance.cl

[(shifts[0,2,1]) != (shifts[1,0,1]),
 (shifts[0,0,0]) != (shifts[0,2,1]),
 (shifts[1,1,0]) != (shifts[1,1,1]),
 (shifts[1,0,1]) != (shifts[1,1,0]),
 (shifts[1,0,0]) != (shifts[1,1,0]),
 (shifts[1,0,1]) != (shifts[1,1,1]),
 (shifts[0,1,0]) != (shifts[0,2,1]),
 (shifts[1,0,0]) != (shifts[1,0,1]),
 (shifts[1,0,0]) != (shifts[1,1,1]),
 (shifts[0,2,1]) != (shifts[1,0,0]),
 (shifts[1,1,0]) != (shifts[1,2,1]),
 (shifts[0,0,0]) != (shifts[0,1,0]),
 (shifts[0,0,0]) != (shifts[0,2,0]),
 (shifts[0,2,0]) != (shifts[1,0,1]),
 (shifts[0,1,0]) != (shifts[0,2,0]),
 (shifts[0,0,1]) != (shifts[0,1,0]),
 (shifts[0,2,0]) != (shifts[0,2,1]),
 (shifts[0,0,1]) != (shifts[0,2,1]),
 (shifts[0,2,0]) != (shifts[1,0,0]),
 (shifts[0,0,0]) != (shifts[0,0,1]),
 (shifts[0,0,1]) != (shifts[0,2,0]),
 (shifts[1,1,1]) != (shifts[1,2,0]),
 (shifts[1,0,0]) != (shifts[1,2,1]),
 (shifts[1,1,0]) != (shifts[1,2,0]),
 (shifts[0,1,1]) != (shifts[0,2,1]),
 (shifts[0,1,1]) != (shifts[0,2,0]),
 (shifts[0,1,0]) != (shifts[0,1,1]),
 

In [10]:
len(learned_instance.cl)

34